In [ ]:
!pip install ezc3d pandas

In [ ]:
!wget https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter1Input.py -O Chapter1Input.py
new_var = %run /content/Chapter1Input.py

In [ ]:
# ============================
# Forces & moments per plate (no zero-clipping; robust vertical detection)
# ============================
import numpy as np
import matplotlib.pyplot as plt

P = c3d["parameters"]

# Rates / sizes
ptHz    = float(P["POINT"]["RATE"]["value"][0])            # 50
anHz    = float(P["ANALOG"]["RATE"]["value"][0])           # 200
nFrames = int(c3d["data"]["points"].shape[2])              # 450
subf    = int(round(anHz / ptHz))                          # 4
t       = np.arange(nFrames, dtype=float) / ptHz

# Analog cube: (subframes, nAnalogChannels, nPointFrames)
A = c3d["data"]["analogs"]

# CHANNEL map: 6 x nPlates (1-based indices into ANALOG)
chan = np.array(P["FORCE_PLATFORM"]["CHANNEL"]["value"]).reshape(6, -1)
nPlates = chan.shape[1]
chan0 = np.clip(chan - 1, 0, A.shape[1]-1)                 # to 0-based, safety-clipped

# Optional CAL_MATRIX (per plate 6x6)
CAL = None
if "CAL_MATRIX" in P["FORCE_PLATFORM"]:
    raw = np.array(P["FORCE_PLATFORM"]["CAL_MATRIX"]["value"])
    if raw.size == 36 * nPlates:
        CAL = raw.reshape(nPlates, 6, 6)
    elif raw.size == 36:
        CAL = np.tile(raw.reshape(1,6,6), (nPlates,1,1))

def collapse_subframes(arr_subf_6_frames):
    """
    Average analog subframes -> point frames.
    Accepts typical shapes: (subf, 6, nFrames), (1, 6, nFrames*subf), etc.
    Returns (6, nFrames).
    """
    arr = np.asarray(arr_subf_6_frames)
    # Common case
    if arr.ndim == 3 and arr.shape[0] == subf and arr.shape[2] == nFrames:
        return arr.mean(axis=0)
    # (1,6,nFrames*subf)
    if arr.ndim == 3 and arr.shape[0] == 1 and arr.shape[2] == nFrames*subf:
        return arr.reshape(1,6,nFrames,subf).mean(axis=3)[0]
    # (subf, nFrames) per channel - reduce
    if arr.ndim == 3 and arr.shape[1] == 6:
        # collapse subframes by mean along axis 0
        return arr.mean(axis=0)
    # Fallback: interpolate each of the 6 channels to point time
    out = []
    for k in range(6):
        y = arr[:, k, :].ravel()
        t_an = np.linspace(0, (len(y)-1)/anHz, len(y))
        out.append(np.interp(t, t_an, y))
    return np.vstack(out)

def pick_vertical_and_flip(F_3xN):
    """
    Pick vertical axis as the one with the largest robust magnitude (95th percentile of |F|).
    Flip sign if the series is mostly negative (so positive is 'up').
    Return (Fx, Fy, Fz_corrected, vert_axis_index).
    """
    mags = [np.percentile(np.abs(F_3xN[i]), 95) for i in range(3)]
    v = int(np.argmax(mags))           # 0=X, 1=Y, 2=Z
    Fv = F_3xN[v].astype(float)
    # Flip if mostly negative (median < 0)
    if np.nanmedian(Fv) < 0:
        Fv = -Fv
        # also flip the chosen axis in the 3xN for consistency
        F = F_3xN.copy()
        F[v] = Fv
    else:
        F = F_3xN
    return F[0], F[1], F[2], v

per_plate = []
for p in range(nPlates):
    # pull the six mapped channels for this plate
    raw6 = A[:, chan0[:, p], :]                         # (subf, 6, nFrames) in lab units
    six_by_frames = collapse_subframes(raw6)            # (6, nFrames)
    # apply CAL if available
    if CAL is not None:
        six_by_frames = (CAL[p] @ six_by_frames).astype(float)

    forces  = six_by_frames[0:3, :]                     # Fx,Fy,Fz (lab coords)
    moments = six_by_frames[3:6, :]                     # Mx,My,Mz

    Fx, Fy, Fz_corr, vert_axis = pick_vertical_and_flip(forces)
    per_plate.append(dict(Fx=Fx, Fy=Fy, Fz=Fz_corr,
                          Mx=moments[0], My=moments[1], Mz=moments[2],
                          vert_axis=vert_axis))

# --------- Plot (compare directly to Mokka) ---------
axis_name = ["X","Y","Z"]
for i, d in enumerate(per_plate, start=1):
    # Forces (no zero-clipping; just flipped if needed)
    plt.figure(figsize=(10,4))
    plt.plot(t, d["Fx"], label="Fx")
    plt.plot(t, d["Fy"], label="Fy")
    plt.plot(t, d["Fz"], label="Fz (vertical)")
    plt.title(f"Plate {i} -- Forces (vertical axis picked: {axis_name[d['vert_axis']]})")
    plt.xlabel("Time (s)"); plt.ylabel("Force (lab units)"); plt.legend(); plt.tight_layout(); plt.show()

    # Moments (as-is)
    plt.figure(figsize=(10,4))
    plt.plot(t, d["Mx"], label="Mx")
    plt.plot(t, d["My"], label="My")
    plt.plot(t, d["Mz"], label="Mz")
    plt.title(f"Plate {i} -- Moments")
    plt.xlabel("Time (s)"); plt.ylabel("Moment (lab units)"); plt.legend(); plt.tight_layout(); plt.show()

    def _pk(v): return float(np.nanmax(np.abs(v))) if v is not None and len(v) else np.nan
    print(f"Plate {i} peaks (abs) -> Fx:{_pk(d['Fx']):.1f}  Fy:{_pk(d['Fy']):.1f}  "
          f"Fz:{_pk(d['Fz']):.1f}   Mx:{_pk(d['Mx']):.1f}  My:{_pk(d['My']):.1f}  Mz:{_pk(d['Mz']):.1f}")
